<a href="https://colab.research.google.com/github/Yahred/evolutionary-computation/blob/main/CGXV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimización de rutas a través de algoritmos genéticos

Se cargan módulos necesarios

In [129]:
pip install deap

ERROR: Could not find a version that satisfies the requirement folium_static (from versions: none)
ERROR: No matching distribution found for folium_static


In [130]:
import json
import math
import random
import folium

import matplotlib.pyplot as plt
import numpy as np

from deap import base, creator, tools, algorithms
from urllib.request import urlopen
from IPython.display import display

Se cargan datos sobre las capitales

In [112]:
path = 'https://raw.githubusercontent.com/Yahred/evolutionary-computation/main/data/capitales.json'
archivo = urlopen(path).read()
data = json.loads(archivo)

CAPITALES = data['capitales']
COORDENADAS = data['coordenadas']
DISTANCIAS = data['distancias']
ESPACIOS = 8
CENTRO_EU = [54.6872, 25.2797]
ZOOM_MAPA = 4

Se calcula la longitud del cromosoma en base a las capitales y el número de espacios disponibles

In [55]:
LONGITUD_CROMOSOMA = 0
for i in range(ESPACIOS):
  LONGITUD_CROMOSOMA += math.ceil(math.log(len(CAPITALES) - i, 2))

LONGITUD_CROMOSOMA

28

In [38]:
CAPITALES

['Madrid',
 'París',
 'Berlín',
 'Roma',
 'Atenas',
 'Lisboa',
 'Varsovia',
 'Praga',
 'Ámsterdam',
 'Viena',
 'Estocolmo',
 'Budapest']

In [39]:
COORDENADAS

{'Madrid': {'latitud': 40.416775, 'longitud': -3.70379},
 'París': {'latitud': 48.856613, 'longitud': 2.352222},
 'Berlín': {'latitud': 52.520008, 'longitud': 13.404954},
 'Roma': {'latitud': 41.902782, 'longitud': 12.496366},
 'Atenas': {'latitud': 37.98381, 'longitud': 23.727539},
 'Lisboa': {'latitud': 38.722252, 'longitud': -9.139337},
 'Varsovia': {'latitud': 52.229676, 'longitud': 21.012229},
 'Praga': {'latitud': 50.075538, 'longitud': 14.4378},
 'Ámsterdam': {'latitud': 52.366696, 'longitud': 4.89454},
 'Viena': {'latitud': 48.208174, 'longitud': 16.373819},
 'Estocolmo': {'latitud': 59.329323, 'longitud': 18.068581},
 'Budapest': {'latitud': 47.497913, 'longitud': 19.040236}}

In [40]:
DISTANCIAS

{'Madrid': {'Madrid': 0,
  'París': 1056,
  'Berlín': 1886,
  'Roma': 2226,
  'Atenas': 2760,
  'Lisboa': 634,
  'Varsovia': 2502,
  'Praga': 1752,
  'Ámsterdam': 1375,
  'Viena': 2275,
  'Estocolmo': 2898,
  'Budapest': 1775},
 'París': {'Madrid': 1056,
  'París': 0,
  'Berlín': 878,
  'Roma': 1105,
  'Atenas': 1842,
  'Lisboa': 1490,
  'Varsovia': 1052,
  'Praga': 1029,
  'Ámsterdam': 431,
  'Viena': 1133,
  'Estocolmo': 1903,
  'Budapest': 1345},
 'Berlín': {'Madrid': 1886,
  'París': 878,
  'Berlín': 0,
  'Roma': 1180,
  'Atenas': 2040,
  'Lisboa': 2154,
  'Varsovia': 265,
  'Praga': 280,
  'Ámsterdam': 657,
  'Viena': 683,
  'Estocolmo': 1618,
  'Budapest': 684},
 'Roma': {'Madrid': 2226,
  'París': 1105,
  'Berlín': 1180,
  'Roma': 0,
  'Atenas': 1584,
  'Lisboa': 1956,
  'Varsovia': 1661,
  'Praga': 1454,
  'Ámsterdam': 1632,
  'Viena': 857,
  'Estocolmo': 2376,
  'Budapest': 1330},
 'Atenas': {'Madrid': 2760,
  'París': 1842,
  'Berlín': 2040,
  'Roma': 1584,
  'Atenas': 0,
  '

Graficamos las capitales que tenemos de opción

In [139]:
def marcar_capitales():
  mapa = folium.Map(location=CENTRO_EU, zoom_start=ZOOM_MAPA)

  for (capital, coord) in COORDENADAS.items():
    lat = coord['latitud']
    lng = coord['longitud']
    folium.Marker([lat, lng], tooltip=capital).add_to(mapa)

  return mapa

marcar_capitales()

# Definición de la función de la evaluación

In [41]:
class Capital:
  def __init__(self, nombre: str, lat, lng) -> None:
    self.nombre = nombre
    self.lat = lat
    self.lng = lng

  def __str__(self):
    return f'{self.nombre} Latitud: {self.lat} Longitud: {self.lng}'

Definimos una función para gráficar el mapa de las capitales

In [166]:
def graficar_capitales(capitales: list[Capital]):
  mapa = marcar_capitales()

  coordenadas = []
  for capital in capitales:
    folium.Marker([capital.lat, capital.lng], tooltip=capital).add_to(mapa)
    coordenadas.append((capital.lat, capital.lng))

  coord = (capitales[0].lat, capitales[0].lng)
  coordenadas.append(coord)

  linea = folium.PolyLine(coordenadas, color='red')
  linea.add_to(mapa)

  return mapa

In [162]:
def capital_factory(nombre: str):
  coordenadas = COORDENADAS[nombre]
  return Capital(nombre, coordenadas['latitud'], coordenadas['longitud'])

In [163]:
def calcular_distancia(a: Capital, b: Capital):
  distancias = DISTANCIAS[a.nombre]
  return distancias[b.nombre]

In [167]:
def calcular_recorrido(capitales: list[Capital]) -> float:
  distancia = 0
  ruta = capitales.copy()

  for i in range(1, len(ruta)):
    a = capitales[i - 1]
    b = capitales[i]
    distancia += calcular_distancia(a, b)

  distancia += calcular_distancia(ruta[-1], ruta[0])

  return distancia

In [159]:
def decode_ind(bin: str) -> list[Capital]:
  opciones = CAPITALES.copy()
  capitales = []
  bin_restante = bin
  for i in range(ESPACIOS):
    n_bits = math.ceil(math.log(len(opciones), 2))
    segmento_bin = bin_restante[:n_bits]
    bin_restante = bin_restante[n_bits:]

    seleccion = int(segmento_bin, 2)

    if seleccion >= len(opciones):
      seleccion = len(opciones) - 1

    capital_seleccionada = capital_factory(opciones.pop(seleccion))
    capitales.append(capital_seleccionada)

  return capitales

In [154]:
def fitness(ind: list[str]):
  capitales = decode_ind(''.join(ind))
  distancia = calcular_recorrido(capitales)
  return distancia,

# Definición de los individuos



In [179]:
def crear_ind():
    return ''.join(random.choice('01') for _ in range(LONGITUD_CROMOSOMA))


creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

toolbox.register("individual", tools.initIterate, creator.Individual, crear_ind)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", fitness)
toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=5)

stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register("avg", np.mean)

/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [180]:
ind = toolbox.individual()
ind

['0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1']

In [178]:
random.seed(64)
n_gen = 10000
initial_pop = 200

pop = toolbox.population(n=initial_pop)
hof = tools.HallOfFame(1)
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.6, mutpb=0.2, ngen=n_gen, halloffame=hof, verbose=True, stats=stats)

ganador = tools.selBest(pop, k=1)[0]

Se truncaron las últimas líneas 5000 del resultado de transmisión.
5001	171   	5198.86
5002	168   	5191.11
5003	170   	5129.84
5004	171   	5028.38
5005	162   	5091.49
5006	164   	5216.18
5007	159   	5083.89
5008	179   	5294.52
5009	179   	5179.39
5010	165   	5222.38
5011	173   	5061.11
5012	160   	5330.39
5013	167   	5117.5 
5014	163   	5075.73
5015	169   	4994.64
5016	163   	5349.7 
5017	168   	5459.88
5018	150   	5212.9 
5019	170   	4991.14
5020	179   	5253.78
5021	172   	5169.41
5022	169   	5032.24
5023	164   	4989.18
5024	176   	5386.48
5025	170   	5385.65
5026	169   	5259.4 
5027	170   	5265.94
5028	169   	5306.73
5029	163   	5488.15
5030	170   	5218.15
5031	169   	5138.85
5032	158   	5242.14
5033	172   	4974.6 
5034	173   	5390.66
5035	168   	5268.56
5036	171   	5180.62
5037	165   	4871.51
5038	172   	5177.96
5039	161   	4895.06
5040	170   	5344.52
5041	162   	5306.09
5042	165   	5407.69
5043	169   	5269.66
5044	173   	5163.47
5045	169   	5087.07
5046	162   	5038.27
5047	158   	5

In [181]:
ganador

['0',
 '0',
 '1',
 '0',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1']

In [182]:
distancia, = fitness(ganador)
distancia

4358

In [183]:
capitales = decode_ind(''.join(ind))

In [184]:
graficar_capitales(capitales)